<div align="center">
    <img src="images/um_logo.png" alt="image">
</div>

## Introducción a la Memoria Compartida en Linux

### Conceptos Fundamentales de IPC (Inter-process Communication)

La comunicación entre procesos (IPC, por sus siglas en inglés) es un mecanismo que permite a los procesos (instancias de programas en ejecución) intercambiar datos y señales. Es fundamental en sistemas operativos que soportan la ejecución concurrente de programas.

Existen varios métodos de IPC, cada uno con sus propias características y usos ideales:
- **Pipes (tuberías)**: Permiten la comunicación unidireccional entre procesos relacionados.
- **Sockets**: Facilitan la comunicación entre procesos en la misma máquina o entre máquinas en una red.
- **Semáforos**: Utilizados para controlar el acceso a recursos compartidos.
- **Colas de mensajes**: Permiten el intercambio de mensajes entre procesos.
- **Memoria compartida**: Proporciona un espacio de memoria que varios procesos pueden acceder directamente.

### Memoria Compartida

La memoria compartida es un espacio de memoria que puede ser accesible por múltiples procesos. Es uno de los métodos más rápidos de IPC porque evita la sobrecarga del kernel al pasar datos directamente entre procesos sin intermediarios. Su eficacia en términos de rendimiento la hace ideal para aplicaciones que requieren un alto rendimiento en comunicación entre procesos, como sistemas de bases de datos y aplicaciones multimedia en tiempo real.

#### Ventajas de la Memoria Compartida
- **Rendimiento**: Es más rápida que otros métodos de IPC porque minimiza las llamadas al sistema.
- **Eficiencia de recursos**: Al compartir una memoria, se minimiza el uso redundante de recursos.

#### Desafíos de la Memoria Compartida
- **Sincronización**: Requiere mecanismos de sincronización para evitar condiciones de carrera y garantizar la consistencia de los datos.
- **Gestión de derechos de acceso**: Es necesario gestionar cuidadosamente quién puede acceder a qué partes de la memoria compartida.

### Implementación en Linux

En Linux, la memoria compartida puede ser implementada usando varias APIs proporcionadas por el sistema operativo, como `shmget`, `shmat`, `shmdt`, y `shmctl`, que son parte de la interfaz System V IPC, o más recientemente a través del estándar POSIX con `mmap`.

#### Conceptos Clave

1. **Segmento de Memoria Compartida**: En el contexto de Linux, un segmento de memoria compartida es una porción de memoria que se designa explícitamente para ser compartida entre varios procesos. Cada segmento tiene un identificador único (clave o key) que los procesos utilizan para acceder al segmento.

2. **Sincronización**: Para manejar el acceso concurrente a la memoria compartida, es necesario emplear mecanismos de sincronización como semáforos o mutexes. Estos mecanismos aseguran que solo un proceso puede acceder a una parte de la memoria a la vez, previniendo así la corrupción de datos.

3. **Persistencia**: A diferencia de otros métodos de IPC, la memoria compartida persiste hasta que sea explícitamente eliminada o hasta que el sistema se reinicie. Esto significa que la información puede persistir entre ejecuciones de diferentes programas, lo que puede ser tanto una ventaja como un riesgo de seguridad.

#### Seguridad y Control de Acceso

Dado que la memoria compartida puede ser accesible por múltiples procesos, el control de acceso se vuelve una consideración importante. En sistemas Unix y Linux, los permisos de acceso a los segmentos de memoria compartida se manejan de manera similar a los archivos:
- **Permisos de lectura (r)**: Permite leer datos del segmento de memoria.
- **Permisos de escritura (w)**: Permite escribir datos en el segmento de memoria.
- **Permisos de ejecución (x)**: Generalmente no aplicable para memoria compartida, pero en contextos de IPC puede representar otras capacidades de control.

### Persistencia de la Memoria Compartida en Linux

La afirmación sobre la persistencia de la memoria compartida en sistemas operativos como Linux es generalmente precisa, pero merece una explicación más detallada para entender completamente su comportamiento y gestión.

#### Memoria Compartida en Linux

En Linux, la memoria compartida es gestionada a través de dos principales mecanismos API: System V IPC y POSIX Shared Memory. La diferencia en el comportamiento de persistencia puede variar ligeramente entre estos dos enfoques:

1. **System V IPC**
   - La memoria compartida creada a través de las llamadas de System V (`shmget`, `shmat`, etc.) no se destruye automáticamente cuando el último proceso que la utiliza termina. En cambio, persiste en el sistema hasta que es explícitamente eliminada con `shmctl` o el sistema se reinicia. Esto proporciona una forma de dejar datos en la memoria que otros procesos pueden acceder más tarde, incluso si el proceso original que creó el segmento de memoria ya no está en ejecución.

2. **POSIX Shared Memory**
   - Similar a System V, la memoria compartida POSIX (creada usando `shm_open` y mapeada con `mmap`) también persiste después de que el proceso que la creó termina. Debe ser explícitamente eliminada mediante `shm_unlink` para removerla completamente del sistema.

#### Implicaciones de Seguridad y Prácticas

La persistencia de la memoria compartida es útil en escenarios donde múltiples aplicaciones, posiblemente reiniciadas o lanzadas en diferentes tiempos, necesitan acceder a un conjunto común de datos rápidamente. Sin embargo, esta característica también implica riesgos de seguridad importantes:

- **Acceso no autorizado**: Si los permisos no están adecuadamente configurados, un proceso no autorizado podría acceder a datos sensibles.
- **Inconsistencias de datos**: Sin mecanismos de sincronización adecuados, varios procesos pueden modificar datos de manera concurrente, llevando a estados inconsistentes.
- **Fugas de recursos**: Si los segmentos de memoria compartida no son eliminados adecuadamente, pueden permanecer consumiendo recursos del sistema indefinidamente.

#### Recomendaciones

Para gestionar eficazmente la memoria compartida, especialmente en un entorno multiusuario o en aplicaciones críticas, es esencial:

- Establecer permisos estrictos para controlar quién puede acceder a la memoria compartida.
- Utilizar mecanismos de sincronización adecuados para evitar condiciones de carrera y asegurar la integridad de los datos.
- Asegurarse de limpiar (eliminar) todos los segmentos de memoria compartida cuando ya no sean necesarios para evitar fugas de recursos y posibles brechas de seguridad.

En conclusión, la persistencia de la memoria compartida es una característica del diseño de IPC en Linux que, aunque útil, requiere un manejo cuidadoso para evitar problemas de seguridad y gestión de recursos.


### Condición de Carrera (Race Condition)

Una condición de carrera ocurre cuando dos o más procesos o hilos de ejecución acceden a recursos compartidos (como memoria, archivos, etc.) y tratan de modificarlos al mismo tiempo. El resultado final de las operaciones depende del orden en que se ejecuten los accesos, lo cual puede variar cada vez que el programa se ejecute. Esto puede llevar a resultados inesperados o errores difíciles de replicar y diagnosticar.

#### Características de una Condición de Carrera

- **No determinismo**: El comportamiento del software puede variar de una ejecución a otra debido a diferencias en el orden de ejecución de las operaciones.
- **Dependencia de la sincronización de procesos/hilos**: Las condiciones de carrera son más comunes en sistemas operativos multitarea y aplicaciones multihilo donde el orden de ejecución no puede ser fácilmente predicho o controlado sin mecanismos de sincronización adecuados.

### Uso de `mmap` en Python para Memoria Compartida

La función `mmap` (memory map) en Python proporciona una manera de acceder a archivos y dispositivos en memoria de una forma eficiente. Utilizando `mmap`, puedes mapear un archivo o dispositivo entero (o una parte) a la memoria, permitiendo que el contenido sea accedido directamente desde el espacio de direcciones de un proceso. Esto es particularmente útil para aplicaciones que necesitan manipular grandes volúmenes de datos sin cargar todo el archivo en la memoria a la vez.

#### Funcionalidades de `mmap`

- **Acceso Eficiente**: Al mapear un archivo a la memoria, los datos pueden ser accedidos más rápidamente que leyendo el archivo en pequeñas partes. Esto es debido a que las operaciones de E/S se minimizan y el acceso se realiza a través de la memoria.
- **Manipulación de Datos**: `mmap` permite tanto leer como modificar datos directamente en la memoria. Los cambios realizados en la memoria se pueden sincronizar con el archivo original.
- **Acceso Aleatorio**: Proporciona la capacidad de acceder y modificar cualquier parte del archivo mapeado sin necesidad de secuencialidad, lo cual es ideal para aplicaciones que requieren acceso aleatorio a grandes conjuntos de datos.

#### Cómo Trabaja `mmap`

`mmap` crea una relación directa entre un objeto de memoria mapeada y un archivo o dispositivo subyacente. Cualquier modificación hecha a la memoria mapeada se refleja en el archivo, y los datos del archivo son accesibles directamente desde la memoria, como si fueran variables normales en el código.

#### Ejemplo de Uso de `mmap` en Python

Aquí te muestro cómo puedes usar `mmap` para leer y modificar un archivo de manera eficiente:

```python
import mmap
import os

# Abre un archivo para lectura y escritura
file_path = 'example.dat'
with open(file_path, 'r+b') as f:
    # Mapea todo el archivo en memoria
    mm = mmap.mmap(f.fileno(), 0)
    
    # Lee algunos bytes del archivo mapeado
    print('Contenido original:', mm[:10].decode('utf-8'))
    
    # Modifica algunos bytes en la memoria
    mm[0:5] = b'Hello'
    
    # Sincroniza los cambios con el archivo original
    mm.flush()
    
    # Lee los cambios del archivo mapeado
    print('Contenido modificado:', mm[:10].decode('utf-8'))
    
    # Cierra el mapeo de memoria
    mm.close()


### Ejemplos Prácticos en Python

````python
"""
Ejemplo que muestra el uso de mmap para utilizarlo desde un único proceso para tener un archivo en memoria.
Correr en modo interactivo. En este archivo se dan comandos para jugar en la consola

"""

import mmap
import os

fd = os.open("lorem.txt", os.O_RDWR)

area = mmap.mmap(fd, 0)

area.read(5)

area.write(b'nuevo')

area.tell()
area.seek(0)
fd
"""
con el comando od se puede ver el archivo en bajo nivel. -c para que muestre los caracteres
"""



In [ ]:
import mmap
import os

# Se define la longitud de la región de memoria a asignar
# mem_size = os.sysconf('SC_PAGE_SIZE') * 10
mem_size = 64
# Se crea un objeto mmap que representa la región de memoria mapeada
mem = mmap.mmap(-1, mem_size, mmap.MAP_ANONYMOUS | mmap.MAP_SHARED)

# Se escribe en la región de memoria compartida
mem.write(b'Hola, esta es una prueba de memoria compartida.')

# Se lee de la región de memoria compartida
mem.seek(0)
data = mem.read(mem_size)

print(data)

# Se libera la región de memoria compartida
mem.close()


### Mapeo de Memoria Anónima en Python

El mapeo de memoria anónima se refiere a la creación de un área de memoria que no está asociada directamente con un archivo en el sistema de archivos. Este tipo de mapeo es útil para compartir datos entre procesos sin necesidad de crear y gestionar archivos físicos. En Python, esto se puede realizar usando la biblioteca `mmap`.

#### Ejemplo de Mapeo de Memoria Anónima:

```python
import mmap
import os

# Se define la longitud de la región de memoria a asignar
# mem_size = os.sysconf('SC_PAGE_SIZE') * 10
mem_size = 64
# Se crea un objeto mmap que representa la región de memoria mapeada
mem = mmap.mmap(-1, mem_size, mmap.MAP_ANONYMOUS | mmap.MAP_SHARED)

# Se escribe en la región de memoria compartida
mem.write(b'Hola, esta es una prueba de memoria compartida.')

# Se lee de la región de memoria compartida
mem.seek(0)
data = mem.read(mem_size)

print(data)

# Se libera la región de memoria compartida
mem.close()


### Explicación del Código

#### Configuración de la Memoria:
- `mem_size` establece el tamaño de la memoria a mapear. Aquí se usa un tamaño fijo de 64 bytes para simplificar el ejemplo.

#### Creación del Objeto Mmap:
- `mmap.mmap(-1, mem_size, mmap.MAP_ANONYMOUS | mmap.MAP_SHARED)` crea un mapeo de memoria anónimo y compartido.
  - `-1` indica que no hay un archivo subyacente.
  - `mem_size` es el tamaño de la memoria.
  - `mmap.MAP_ANONYMOUS | mmap.MAP_SHARED` son banderas que especifican que el mapeo es anónimo y compartido entre procesos.

#### Operaciones de Escritura y Lectura:
- `mem.write(...)`: Escribe una cadena de bytes en la memoria mapeada.
- `mem.seek(0)`: Reposiciona el puntero al inicio de la memoria para la lectura.
- `mem.read(mem_size)`: Lee el contenido de la memoria mapeada.

#### Cierre y Limpieza:
- `mem.close()`: Cierra el objeto `mmap` y libera el área de memoria asociada, asegurando que no haya fugas de memoria.

### Implicaciones y Usos
El mapeo de memoria anónima es especialmente útil en situaciones donde varios procesos necesitan acceder y modificar rápidamente un conjunto común de datos sin la intervención de operaciones de disco, lo que mejora significativamente la velocidad y eficiencia de la comunicación interprocesos.
